## Solution of 5.9.1, Bee Checklist

First of all, we import the two modules we'll need to read the `csv` file, and to use regular expressions:

In [1]:
import csv
import re

Then, we read the file, and store the columns `Scientific Name` and `Taxon Author` in two lists:

In [2]:
with open('../data/bee_list.txt') as f:
    csvr = csv.DictReader(f, delimiter = '\t')
    species = []
    authors = []
    for r in csvr:
        species.append(r['Scientific Name'])
        authors.append(r['Taxon Author'])

How many species?

In [3]:
len(species)

19508

In [4]:
len(authors)

19508

Pick one of the `authors` element to use for testing. Choose one that is quite complicated, such as the 38th element:

In [5]:
au = authors[37]

In [6]:
au

'Tadauchi, Hirashima & Matsumura, 1987'

Now we need to build a regular expression. After some twiddling, you should end up with something like this, which captures the authors in one group, and the year in another group:

In [7]:
my_reg = re.compile(r'\(?([\w\s,\.\-\&]*),\s(\d{4})\)?')
# Translation
# \(? -> open parenthesis (or not)
# ([\w\s,\.\-\&]+) -> the first group is the list of authors
#                     which can contain \w (word character)
#                     \s (space) \. (dot) \- (dash) \& (ampersand)
# ,\s -> followed by comma and space
# (\d{4}) -> the second group is the year, 4 digits
#  \)? -> potentially, close parenthesis

Test the expression

In [8]:
re.findall(my_reg,au)

[('Tadauchi, Hirashima & Matsumura', '1987')]

Now we write a function that uses the regular expression to extract an author list (useful when there are multiple authors), and the year

In [9]:
def extract_list_au_year(au):
    tmp = re.match(my_reg, au)
    authorlist = tmp.group(1)
    year = tmp.group(2)
    # split authors into a list using re.split
    authorlist = re.split(', | \& ', authorlist)
    # Translation: either separate using ', ' or ' & '
    return [authorlist, year]

Let's see the output of this function:

In [10]:
extract_list_au_year(au)

[['Tadauchi', 'Hirashima', 'Matsumura'], '1987']

Finally, let's build two dictionaries:
- one tracking the number of times each year is mentioned in the database;
- one traking the number of times each author is mentioned

In [11]:
dict_years = {}
dict_authors = {}
for au in authors:
    tmp = extract_list_au_year(au)
    for aunum in tmp[0]:
        if aunum in dict_authors.keys():
            dict_authors[aunum] = dict_authors[aunum] + 1 
        else:
            dict_authors[aunum] =  1 
    if tmp[1] in dict_years.keys():
        dict_years[tmp[1]] = dict_years[tmp[1]] + 1 
    else:
        dict_years[tmp[1]] =  1 

For example, these are all the authors:

In [12]:
dict_authors

{'Michener': 455,
 'Cockerell': 3394,
 'Timberlake': 864,
 'Zavortink': 7,
 'Dours': 62,
 'Eversmann': 41,
 'Pérez': 309,
 'Warncke': 507,
 'Hirashima': 210,
 'Viereck': 130,
 'Morawitz': 513,
 'LaBerge': 192,
 'Friese': 1330,
 'Smith': 943,
 'Scopoli': 8,
 'Thorp': 19,
 'Osytshnjuk': 74,
 'Tadauchi': 76,
 'Matsumura': 19,
 'Popov': 86,
 'Alfken': 149,
 'Radoszkowski': 113,
 'Rossi': 11,
 'Perkins': 65,
 'Robertson': 135,
 'Benoist': 164,
 'Miyanaga': 2,
 'Dawut': 1,
 'Lebedev': 9,
 'Stoeckhert': 10,
 'Linsley': 59,
 'MacSwain': 16,
 'Cresson': 437,
 'Kirby': 74,
 'Ribble': 31,
 'Cameron': 158,
 'Blüthgen': 315,
 'Kreichbaumer': 1,
 'Scheuchl': 10,
 'Gusenleitner': 39,
 'Schmiedeknecht': 50,
 'Mitchell': 270,
 'Hedicke': 24,
 'Xu': 37,
 'Donovan': 24,
 'Nurse': 53,
 'Malloch': 3,
 'Bouseman': 8,
 'Panzer': 29,
 'Schulthess': 1,
 'Fabricius': 138,
 'Vachal': 557,
 'Larkin': 6,
 'Stephens': 1,
 'Bingham': 47,
 'Wu': 213,
 'Dalla Torre': 65,
 'Strand': 249,
 'Casad': 7,
 'Pittioni': 7,
 '

### What is the name of the author with most entries in the database?


We use the following strategy:
- we find the maximum value in the dictionary
- we use the function index to find to which entry is it associated
- we find the corresponding author

In [13]:
max_value_author = max(dict_authors.values())
max_value_author

3394

In [14]:
which_index = list(dict_authors.values()).index(max_value_author)
which_index

1

An the winner is:

In [15]:
list(dict_authors.keys())[which_index]

'Cockerell'

### Which year of publication is most represented in the database?

We use the same strategy to find that the golden year of bee publication is:

In [16]:
max_value_year = max(dict_years.values())
which_index = list(dict_years.values()).index(max_value_year)
list(dict_years.keys())[which_index]

'1903'